# Iterate on integer

We want following code works, but it raise `TypeError`:

In [2]:
for i in 5:
    print(i)

<class 'TypeError'>: 'int' object is not iterable

Patch the system with `patch_int_iter()`:

In [25]:
import sys
import cffi
import ctypes
from ctypes import CFUNCTYPE, c_ssize_t, c_void_p

def int_iter(obj_addr):
    obj = ctypes.cast(obj_addr, ctypes.py_object).value
    iter_obj = iter(range(obj))
    ctypes.pythonapi.Py_IncRef(id(iter_obj))
    return id(iter_obj)

def patch_int_iter():
    tp_iter_offset = 216
    if sys.maxsize == 0x7fffffff:
        tp_iter_offset //= 2

    ffi = cffi.FFI()
    ctypes.pythonapi.Py_IncRef.argtypes = [ctypes.c_size_t]

    ITER_FUNC = CFUNCTYPE(c_ssize_t, c_ssize_t)
    cint_iter = ITER_FUNC(int_iter)

    tp_iter_pointer = ffi.cast("size_t *", id(int) + 216//2)
    tp_iter_pointer[0] = ctypes.cast(cint_iter, c_void_p).value

patch_int_iter()

and it does the magic:

In [26]:
for i in 5:
    print(i)

0
1
2
3
4


## How it works

Every type is an instance of `PyTypeObject`:

```c
typedef struct _typeobject {
    PyObject_VAR_HEAD
    const char *tp_name; /* For printing, in format "<module>.<name>" */
    //...

    /* Iterators */
    getiterfunc tp_iter; // <- We need the offset of this field
    iternextfunc tp_iternext;
    //...
} PyTypeObject;
```

When call `iter` on an object, the `tp_iter` field of the type object will be called `PyObject_GetIter()`:

```c
PyObject *
PyObject_GetIter(PyObject *o)
{
    PyTypeObject *t = o->ob_type;
    getiterfunc f = NULL;
    f = t->tp_iter;
    if (f == NULL) {
        if (PySequence_Check(o))
            return PySeqIter_New(o);
        return type_error("'%.200s' object is not iterable", o);
    }
    else {
        PyObject *res = (*f)(o);
        if (res != NULL && !PyIter_Check(res)) {
            PyErr_Format(PyExc_TypeError,
                         "iter() returned non-iterator "
                         "of type '%.100s'",
                         res->ob_type->tp_name);
            Py_DECREF(res);
            res = NULL;
        }
        return res;
    }
}
```

We can use the C compiler to get the offset of `tp_iter`:

```python
import cffi
ffi = cffi.FFI()
ffi.cdef("""
size_t tp_iter_offset;
""")

lib = ffi.verify("""
size_t tp_iter_offset = offsetof(PyTypeObject, tp_iter);
""")
```

the offset is 216 on 64 bit system, and 108 on 32 bit system.